In [17]:
from pathlib import Path

In [18]:
BASE_DIR = Path("..").resolve()
print(BASE_DIR)

clapnq_path =  BASE_DIR/"history_selected_rewrite_queries/clapnq_rewritten_last_turn_qwen3_30B.jsonl"
cloud_path = BASE_DIR/"history_selected_rewrite_queries/cloud_rewritten_last_turn_qwen3_30B.jsonl"
fiqa_pth = BASE_DIR/"history_selected_rewrite_queries/fiqa_rewritten_last_turn.jsonl"
govt_path = BASE_DIR/"history_selected_rewrite_queries/govt_rewritten_last_turn.jsonl"
paths = [clapnq_path, cloud_path, fiqa_pth, govt_path]

/Users/sandylin/Challenge in CL


In [16]:
with open(BASE_DIR/"history_selected_rewrite_queries/all_last_turn.jsonl", "w") as fout:
    for path in paths:
        with open(path, "r") as fin:
            for line in fin:
                fout.write(line)

In [20]:
eval_data = BASE_DIR/"queries data/rag_taskAC.jsonl"
print(eval_data)

/Users/sandylin/Challenge in CL/queries data/rag_taskAC.jsonl


In [22]:
import json
def read_data(path):
    with open(path, "r") as file:
        return file.readlines()

dataset = read_data(eval_data)
print(dataset[1])

{"conversation_id": "b3b321e9ea81d1d90e528f85fff72d63", "task_id": "b3b321e9ea81d1d90e528f85fff72d63<::>8", "task_type": "rag", "turn": "8", "dataset": "MT-RAG 2.0 Authors (Internal)", "input": [{"speaker": "user", "text": "Can you summarize the differences between good bots and bad bots?", "metadata": {"author_type": "human", "author_id": "fd309ff8-8b41-49a0-b483-2fee97472e71", "created_at": 1723641766}}, {"speaker": "agent", "text": "Bots are software programs that typically use a \"headless browser\" to make HTTP requests and other actions. A bot that performs a helpful service can be considered \"good.\" For example, customer service chatbots, search engine crawlers, and performance monitoring bots are generally good bots. Good bots look for and abide by the rules outlined in a website's robots.txt file and read them and avoid that webpage. On the other hand, bad bots range from malicious to simply annoying; for instance, breaking into user accounts to steal data or buying concert 

In [28]:
questions = {}
for data in dataset:
    obj = json.loads(data)
    task_id = obj["task_id"]
    cov = obj["input"]
    all_questions = []
    for q in cov:
        if q["speaker"] == "user":
            all_questions.append(q["text"])
    questions[task_id] = {"_id": task_id, "text": all_questions}

print(questions)
    

{'18ef26058d321c5d96ca3ebf8117789e<::>7': {'_id': '18ef26058d321c5d96ca3ebf8117789e<::>7', 'text': ['How to pay with cash when car shopping?', 'Or installment easier?', 'Then, 0% interest same as having benefits of the both?', "If my friend paid for me with her bank's check, what is a consequence?", 'Do I really need an extra warranty on my new car?', 'EV used car market', "I mean current EV's battery does not stand for a used car market...how do you think?"]}, 'b3b321e9ea81d1d90e528f85fff72d63<::>8': {'_id': 'b3b321e9ea81d1d90e528f85fff72d63<::>8', 'text': ['Can you summarize the differences between good bots and bad bots?', 'What are the features in the CIS bot?', 'How does CIS manage bots?', 'How can I update machine leaning models?', 'Can I create a machine learning model and how should I do that?', 'Speaking about bots again, can they get private information from the user', 'By the way, what is a secret?', 'I heard the toolchain is not available in South America.']}, '720131905932

In [30]:
output_path = BASE_DIR/"queries data/rag_taskAC_all_queries.jsonl"
with open(eval_data, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        obj = json.loads(line)
        task_id = obj["task_id"]
        cov = obj["input"]
        all_questions = []
        for q in cov:
            if q["speaker"] == "user":
                all_questions.append(q["text"])
        robj = {"_id": task_id, "text": "|user|:"+"\n|user|:".join(all_questions)}
        fout.write(json.dumps(robj) + "\n")
        

In [36]:
REF_DIR = BASE_DIR/"eval_script/rag_taskAC_all_queries.jsonl"         
REWRITE_PATH = BASE_DIR/("eval_script/TaskAC_all_queries_rewritten.jsonl") 

def load_ids(path, id_key):
    ids = set()
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                continue
            if id_key in obj:
                ids.add(obj[id_key])
    return ids

# 原始 queries 的 task_id
ref_ids = load_ids(REF_DIR, "_id")

# rewrite 檔裡的 _id
rewrite_ids = load_ids(REWRITE_PATH, "_id")

# 找 missing
missing_ids = sorted(ref_ids - rewrite_ids)

print(f"Missing rewrite count: {len(missing_ids)}")
print("Missing IDs:")
for mid in missing_ids:
    print(mid)



Missing rewrite count: 0
Missing IDs:
